# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмите остаток от деления на 4 и прибавьте 1.

## 3 вариант 

In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000, random_state=686)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.2900575709000693e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [4]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001, random_state=686).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.744985494322756e-12


array([ 4.76783042e-08, -4.53842273e-08, -2.36903536e-08, -1.32251825e-09,
        4.75085869e-08,  3.57233559e-08,  4.57344264e-08, -3.14619725e-08,
        2.34091496e-09,  8.01573064e-09,  3.81918705e+01, -1.96456194e-08,
       -1.70073003e-08, -2.76973133e-08,  8.41441400e-08,  5.80866527e-08,
       -2.70180968e-08,  1.91161239e-08,  1.37219170e-08, -2.68484909e-08,
       -1.57405107e-08,  1.20871593e-08, -3.95256327e-08, -1.70886247e-08,
       -2.86745159e-08,  1.43282656e-08,  1.34941028e-08, -4.72163875e-08,
        2.61457594e-08, -1.73034771e-08,  3.51196502e-08,  3.66861472e-08,
       -5.23528532e-08,  1.48429211e-08,  1.53621545e-08,  4.25533222e-08,
       -2.63555936e-08, -5.40402157e-08,  1.91010779e-08,  4.86898160e-08,
        2.07871111e+01, -3.68272019e-08,  6.29339627e-08, -1.86099695e-08,
       -1.26148609e-08, -4.77004073e-08, -4.68199665e-08, -3.63329883e-09,
       -6.75544743e-08,  2.22005612e-08,  7.62186312e-08, -4.25714429e-09,
       -1.26620626e-08,  

### Задание 1 (0.5 балла)

**Объясните, чем вызвано различие двух полученных значений метрики?**

 Значение метрики при обучение линейной регрессии, полученное с использованием аналитической формулы: $1.2900575709000693e-25$

 Значение метрики при обучение линейной регрессии методом градиентного спуска : $3.744985494322756e-12$

$$1.2900575709000693e-25 < 3.744985494322756e-12$$

  Метод стохастического градиентного спуска на каждой итерации выбирает один случайный объект и сдвигается в сторону антиградиента функции потерь по этому объекту - то есть в направлении уменьшении ошибки по этому объекту. При грамотно подобранной величине стохастического градиентного шага через некоторое число шагов мы попадем в некоторую малую окрестность точки минимума. Однако гипермараметры могут быть подобраны недостаточно хорошо, что не позволит гарантированно достичь самой точки минимума, полученной аналитическим решением. В свою очередь аналитическое решение однозначно находит эту точку минимума. 
  

### Задание 2 (0.5 балла)

**Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression**

In [5]:
from tqdm.auto import tqdm

alphas = 1 / 10 ** (np.arange(19))
penalties = ['l2', 'l1', 'elasticnet']

best_alpha = 0
best_penalty = 'l2'
best_mse = 100

for alpha in tqdm(alphas):
    for penalty in penalties:
        reg = SGDRegressor(alpha=alpha, penalty=penalty, random_state=686).fit(X, y)
        current_mse = mean_squared_error(y, reg.predict(X))
        if current_mse < best_mse:
            best_alpha = alpha
            best_penalty = penalty
            best_mse = current_mse

print('Лучшая alpha:', best_alpha)
print('Лучшee Penalty:', best_penalty)
print('Лучшeе MSE:',best_mse)


Лучшая alpha: 1e-16
Лучшee Penalty: l1
Лучшeе MSE: 7.623786944622722e-25


### Альтернативный способ поиска через GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

reg = SGDRegressor(random_state=686)
num_splits = 5
alphas = 1 / 10 ** (np.arange(19))
penalties = ['l2', 'l1', 'elasticnet']

best_alpha = 0
best_penalty = 'l2'
best_mse = 100



params = {'alpha':alphas,
         'penalty':penalties,
          }
    
    
# print(params)
cv = GridSearchCV(reg,
                  params,
                  scoring='neg_mean_squared_error',
                  cv=num_splits, 
                  n_jobs=-1
                 )
cv.fit(X, y)

print(cv.best_params_)

reg = SGDRegressor(**cv.best_params_, random_state=686)
reg.fit(X, y)
print('MSE:', mean_squared_error(y, reg.predict(X)))

{'alpha': 1e-15, 'penalty': 'l1'}
MSE: 8.239927115683944e-25


Напомним, полученное ранее значение (аналитически) MSE было: $1.2900575709000693e^{−25}$

Полученное лучшее значение MSE при подборе гиперпараметров (согласно первому методу поиска): $7.623786944622722e^{-25}$ 

при альтернативном варианте: $8.239927115683944e^{-25}$

## Ваша многомерная линейная регрессия

### Задание 3 (5 баллов)

**Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом градиентного спуска. Для этого используйте шаблонный класс.**

Критерий останова: либо **норма разности весов на текущей и предыдущей итерациях меньше определенного значения** (первый и **третий варианты**), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.



In [7]:
from statsmodels.tools.tools import add_constant

class LinearRegression(object):
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        self.w = np.random.randn(X.shape[1]+1) 
        X = add_constant(X)

        for i in tqdm(range(self.max_iter)):
            grad = -2 / X.shape[0] * X.T @ (y - X @ self.w) 
            self.w -= self.alpha * grad
            
            if np.linalg.norm(self.alpha * grad) < self.tol:
                break
                
        # self.w = np.linalg.inv(X.T @ X) @ X.T @ y
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        X = add_constant(X)
        return X @ self.w

In [8]:
my_reg = LinearRegression(alpha=0.04, l_ratio=0.0001, tol=1e-5, max_iter=1000)
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')


You are amazing! Great work!


In [9]:
mse_linear = mean_squared_error(y, my_reg.predict(X))

In [10]:
print ('Оптимизированная MSE методом градиентного спуска:', mean_squared_error(y, my_reg.predict(X)))

Оптимизированная MSE методом градиентного спуска: 1.4833032292516285e-08


### Задание 4 (2 балла)

Добавьте l1 (первый и второй варианты) или **l2** (**третий** и четвертый варианты) регуляризацию.

In [11]:
from statsmodels.tools.tools import add_constant

class LinearRegression(object):
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        self.w = np.random.randn(X.shape[1]+1) 
        X = add_constant(X)

        for i in tqdm(range(self.max_iter)):
#             grad = -2 / X.shape[0] * X.T @ (y - X @ self.w) 
            grad = (-2 / X.shape[0]) * X.T @ (y - X @ self.w) + 2 * self.l_ratio * self.w
            self.w -= self.alpha * grad
            
            if np.linalg.norm(self.alpha * grad) < self.tol:
                break
                
        # self.w = np.linalg.inv(X.T @ X) @ X.T @ y
   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        X = add_constant(X)
        return X @ self.w

In [12]:
my_reg = LinearRegression(alpha=0.04, l_ratio=0.0001, tol=1e-5, max_iter=10000)
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')


You are amazing! Great work!


In [13]:
mean_squared_error(y, my_reg.predict(X))

0.0003766660421207986

In [14]:
print ('MSE без регуляризации L2:',mse_linear)
print ('MSE c регуляризацией  L2:', mean_squared_error(y, my_reg.predict(X)))

MSE без регуляризации L2: 1.4833032292516285e-08
MSE c регуляризацией  L2: 0.0003766660421207986


### Задание 5 (1 балл)

**Обучите линейную регрессию из коробки**

* с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и второй вариант) или с **l2-регуляризацией (from sklearn.linear_model import Ridge, третий** и четвертый вариант)
* со значением параметра регуляризации **0.1** - для первого и **третьего варианта**, 0.01 - для второго и четвертого варианта. 

**Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.**

В случае линеной регрессии, когда модель переобучается - те становится слишком специфичной и теряет обобщающую способность, используют регуляризацией для борьбы с переобучением соотвественно

Регуляризация Ridge или L2-регуляризация:

$Q_{ridge}(w) = Q(w) + \alpha \Sigma_{j=0}^{k}w_{k}^{2}$


Несмотря на то, что оба вида регуляризации ведут к занижению значений весов, в случае Ridge регрессии веса могут быть сколько угодно близки к 0, но никогда не равны.

In [15]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(X, y)
mse_sklearn = mean_squared_error(y, ridge.predict(X))
mse_sklearn


3.7388791425941274e-06

In [16]:
my_reg = LinearRegression(alpha=0.01, max_iter=10000, tol=1e-6, l_ratio=0.1)
my_reg.fit(X, y)
mean_squared_error(y, my_reg.predict(X))

307.8287498768968

Была попытка посмотреть с меньшим шагом и с большим количеством итераций соответсвенно (прогонял полчаса)-результат не изменился

In [17]:
print (' MSE Ridge:',mean_squared_error(y, ridge.predict(X)) )
print (' MSE Linear regression:',mean_squared_error(y, my_reg.predict(X)) )

 MSE Ridge: 3.7388791425941274e-06
 MSE Linear regression: 307.8287498768968


По линейной регресии - чем больше параметр регуляризации - тем более сильно искажен градиент (посмотрим формулу): $$grad = (-2 / X.shape[0]) * X.T @ (y - X @ self.w) + 2 * self.lratio * self.w$$

Это может привести к тому, что спуск не достигнет точки минимума - может и вовсе уйти в сторону в связи с искажением градиента. 

Поэтому получаем качество метрики соотвественно хуже. Необходимо взять параметр регуляризации менее 0.1, что позволит снизить искажение градиента и достичь ошибку меньше соотвественно. 

In [18]:
my_reg = LinearRegression(alpha=0.01, max_iter=10000, tol=1e-5, l_ratio=0.01)
my_reg.fit(X, y)
mean_squared_error(y, my_reg.predict(X))

3.664947022503247

***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

*загружен в anytask*

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

 *загружен в anytask*

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.